In [1]:
from sklearn.datasets import load_iris
import pandas as pd
from psyke.utils.dataframe import get_discrete_features_supervised, get_discrete_dataset
from psyke.utils.logic import pretty_theory
from psyke import Extractor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [2]:
x, y = load_iris(return_X_y=True, as_frame=True)
x.columns = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
y = pd.DataFrame(y).replace({"target": {0: 'setosa', 1: 'virginica', 2: 'versicolor'}})
y.columns = ['iris']

dataset = x.join(y)

iris_features = get_discrete_features_supervised(dataset)
x = get_discrete_dataset(x, iris_features)

discretized = x.join(y)

In [3]:
_, original_test = train_test_split(dataset, test_size=0.5, random_state=1)
train, test = train_test_split(discretized, test_size=0.5, random_state=1)

predictor = KNeighborsClassifier(n_neighbors=7)
predictor.fit(train.iloc[:, :-1], train.iloc[:, -1])

extractor = Extractor.real(predictor, iris_features)
theory = extractor.extract(train)
print('Extracted rules:\n\n' + pretty_theory(theory))

Extracted rules:

iris(PetalLength, PetalWidth, SepalLength, SepalWidth, setosa) :-
    PetalWidth =< 0.6474, SepalLength =< 5.3892, PetalLength not_in [2.2685, 4.865].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, setosa) :-
    SepalLength in [5.3892, 6.262], SepalWidth > 3.201, PetalLength not_in [2.2685, 4.865], PetalLength =< 4.865, PetalWidth not_in [0.6474, 1.6376], PetalWidth =< 1.6376.
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, versicolor) :-
    PetalLength > 4.865, SepalLength > 6.262, PetalWidth > 0.6474, SepalWidth > 2.872.
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, versicolor) :-
    PetalWidth > 1.6376, PetalLength > 2.2685, PetalLength not_in [2.2685, 4.865].
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    SepalWidth =< 2.872, PetalLength > 2.2685, PetalLength =< 4.865, PetalWidth > 0.6474, PetalWidth =< 1.6376.
iris(PetalLength, PetalWidth, SepalLength, SepalWidth, virginica) :-
    PetalLength > 2.2685, PetalLe

In [4]:
from psyke.utils.ontology import dataframe_to_ontology, rules_from_theory, individuals_from_dataframe
from owlready2 import *
owlready2.JAVA_EXE = "C:\\Program Files (x86)\\Common Files\\Oracle\\Java\\javapath\\java.exe"

onto = dataframe_to_ontology(dataset)
individuals = individuals_from_dataframe(onto, original_test.iloc[:, :-1])
onto = rules_from_theory(onto, theory, dataset)

sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True, debug = 0)
#for individual, output in zip(individuals, original_test.iloc[:, -1]):
#    print(individual.iris, output)

print(list(onto.rules()))

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


[Iris(?iris), PetalLength(?iris, ?PetalLength), PetalWidth(?iris, ?PetalWidth), SepalLength(?iris, ?SepalLength), SepalWidth(?iris, ?SepalWidth), greaterThan(?SepalLength, 5.3892), lessThanOrEqual(?SepalWidth, 2.872), greaterThan(?PetalWidth, 0.6474), lessThanOrEqual(?PetalWidth, 1.6376) -> iris(?iris, 'virginica'), Iris(?iris), PetalLength(?iris, ?PetalLength), PetalWidth(?iris, ?PetalWidth), SepalLength(?iris, ?SepalLength), SepalWidth(?iris, ?SepalWidth), lessThanOrEqual(?SepalWidth, 3.201), lessThanOrEqual(?SepalLength, 6.262), greaterThan(?PetalLength, 2.2685), greaterThan(?PetalWidth, 0.6474), lessThanOrEqual(?PetalWidth, 1.6376) -> iris(?iris, 'virginica'), Iris(?iris), PetalLength(?iris, ?PetalLength), PetalWidth(?iris, ?PetalWidth), SepalLength(?iris, ?SepalLength), SepalWidth(?iris, ?SepalWidth), lessThanOrEqual(?SepalWidth, 3.201), greaterThan(?PetalWidth, 0.6474), greaterThan(?SepalLength, 5.3892), lessThanOrEqual(?SepalLength, 6.262), greaterThan(?PetalLength, 2.2685), les